In [1]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm

import torch.backends.cudnn as cudnn
import random

## Claim: the data preprocessing part is from https://www.kaggle.com/code/pavansanagapati/a-simple-cnn-model-beginner-guide/notebook

In [2]:
# This code is from https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy

class Data(Dataset):
    
    def __init__(self, data, transform = None):
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
             # first column is of labels.
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [3]:
train_csv = pd.read_csv("fashion-mnist_train.csv")
test_csv = pd.read_csv("fashion-mnist_test.csv")

train_data = Data(train_csv, transform=transforms.Compose([transforms.ToTensor()]))
test_data = Data(test_csv, transform=transforms.Compose([transforms.ToTensor()]))

batch_size = 256
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# From now on, we start to build our model
# We are also really big sad that we can't use maxpool inside sequential

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # image shape is 28 * 28 * 1, where 1 is one color channel
        # 28 * 28 is the image size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        # output shape = (28 - 3 + 1) * (28 - 3 + 1) * 3 = 26 * 26 * 32
        # maxpooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # output shape = 13 * 13 * 32
        # Note the volumn depth is not changed

        self.dropout = nn.Dropout(p=0.25)
        # output shape = 13 * 13 * 32


        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        # output shape = (13 - 3 + 1) * (13 - 3 + 1) * 64 = 11 * 11 * 64

        # maxpooling
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # output shape = 5 * 5 * 64

        self.dropout = nn.Dropout(p=0.25)


        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        # output shape = (5 - 3 + 1) * (5 - 3 + 1) * 128 = 3 * 3 * 128

        self.dropout = nn.Dropout(p=0.4)
        # output shape = 3 * 3 * 128
        
        # fully connected layer
        self.fc1 = nn.Linear(3*3*128, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        # first conv
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = F.relu(self.conv3(x))
        x = self.dropout(x)
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)

        # fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.softmax(self.fc2(x))
        return x

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ConvNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [6]:
model.train()
epoches = 40
lost = []
for epoch in tqdm(range(epoches)):
    train_loss, correct = 0, 0
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)
        
        pred = model(X)

        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # record loss
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    size = len(train_dataloader.dataset)
    train_loss /= len(train_dataloader)
    correct /= size
    print(f" Train accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f}")
    lost.append(train_loss)


  2%|▎         | 1/40 [00:21<14:16, 21.97s/it]

 Train accuracy: 33.6%, Avg loss: 2.122391


  5%|▌         | 2/40 [00:43<13:39, 21.56s/it]

 Train accuracy: 47.8%, Avg loss: 1.982331


  8%|▊         | 3/40 [01:04<13:10, 21.38s/it]

...

 Train accuracy: 84.6%, Avg loss: 1.614141


 98%|█████████▊| 39/40 [35:24<00:24, 24.94s/it]

 Train accuracy: 84.8%, Avg loss: 1.613106


100%|██████████| 40/40 [35:45<00:00, 53.65s/it]

 Train accuracy: 85.0%, Avg loss: 1.611226


In [7]:
# testing set

model.eval()
correct = 0
# Turn off gradient descent
with torch.no_grad():
    for X, y in tqdm(test_dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        print("y_true: ", y, "y_pred: ", pred)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
size = len(test_dataloader.dataset)
correct = correct / size
print(f" Test accuracy: {(100*correct):>0.1f}%")

 10%|█         | 4/40 [00:00<00:00, 38.47it/s]

y_true:  tensor([0, 1, 2, 2, 3, 2, 8, 6, 5, 0, 3, 4, 4, 6, 8, 5, 6, 3, 6, 4, 4, 4, 2, 1,
        5, 7, 8, 4, 4, 1, 5, 7, 7, 8, 1, 0, 9, 8, 0, 8, 2, 0, 4, 6, 2, 0, 3, 3,
        2, 3, 2, 2, 9, 3, 0, 9, 9, 4, 6, 0, 4, 5, 4, 6, 1, 1, 0, 9, 5, 2, 7, 3,
        4, 6, 5, 7, 1, 6, 1, 4, 9, 8, 1, 2, 4, 8, 9, 4, 1, 6, 3, 4, 2, 2, 2, 6,
        4, 7, 7, 3, 9, 3, 9, 0, 8, 2, 3, 8, 2, 7, 5, 5, 3, 2, 7, 5, 0, 2, 7, 1,
        0, 5, 4, 4, 7, 0, 8, 5, 0, 3, 1, 7, 9, 4, 9, 6, 4, 4, 2, 4, 3, 3, 3, 2,
        2, 6, 0, 0, 1, 3, 4, 3, 3, 1, 9, 3, 3, 3, 9, 5, 6, 7, 7, 3, 2, 4, 0, 8,
        7, 2, 2, 8, 9, 0, 2, 4, 4, 5, 7, 9, 9, 1, 3, 9, 1, 5, 5, 6, 0, 7, 4, 9,
        1, 6, 0, 0, 0, 4, 0, 9, 0, 4, 2, 5, 5, 8, 6, 2, 1, 9, 0, 4, 7, 1, 9, 5,
        9, 0, 2, 8, 5, 7, 7, 3, 2, 4, 5, 7, 8, 1, 9, 5, 6, 2, 9, 7, 4, 0, 9, 2,
        1, 5, 7, 7, 0, 2, 4, 5, 3, 3, 8, 1, 6, 2, 4, 8]) y_pred:  tensor([[1.0000e+00, 2.1949e-38, 1.0824e-25,  ..., 0.0000e+00, 2.5395e-33,
         2.3822e-44],
        [2.4177e-27, 1.0000e

 22%|██▎       | 9/40 [00:00<00:00, 41.37it/s]

y_true:  tensor([4, 6, 9, 9, 7, 5, 6, 1, 3, 1, 3, 2, 5, 7, 1, 4, 1, 5, 4, 4, 4, 7, 4, 7,
        1, 5, 6, 4, 2, 6, 9, 7, 0, 7, 9, 3, 2, 1, 7, 0, 9, 6, 8, 8, 5, 4, 8, 2,
        1, 7, 7, 8, 6, 7, 5, 0, 3, 5, 6, 7, 0, 0, 7, 4, 4, 6, 8, 0, 1, 6, 5, 3,
        3, 0, 4, 8, 3, 9, 8, 9, 2, 7, 9, 5, 0, 4, 9, 6, 2, 8, 4, 1, 4, 5, 8, 1,
        7, 1, 1, 6, 8, 8, 3, 9, 4, 5, 4, 6, 7, 3, 3, 6, 2, 4, 2, 6, 2, 1, 6, 8,
        1, 2, 3, 0, 1, 4, 2, 0, 3, 5, 6, 6, 4, 2, 4, 3, 1, 1, 7, 5, 9, 6, 4, 7,
        3, 3, 3, 3, 7, 0, 1, 7, 7, 0, 5, 0, 7, 8, 2, 4, 7, 4, 6, 4, 9, 1, 4, 4,
        6, 2, 4, 0, 2, 1, 9, 0, 3, 8, 6, 1, 3, 4, 8, 9, 6, 9, 6, 2, 5, 9, 7, 4,
        5, 6, 2, 6, 6, 8, 8, 5, 6, 8, 0, 0, 7, 9, 5, 9, 0, 5, 3, 3, 9, 6, 2, 5,
        8, 6, 7, 7, 0, 9, 9, 8, 6, 7, 0, 6, 2, 5, 0, 8, 4, 3, 9, 3, 0, 3, 2, 1,
        1, 8, 9, 0, 0, 7, 4, 1, 6, 2, 6, 2, 4, 1, 1, 4]) y_pred:  tensor([[3.1787e-12, 1.3790e-10, 9.9903e-01,  ..., 5.5328e-14, 5.6327e-11,
         2.8533e-12],
        [3.0861e-17, 8.9137e

 35%|███▌      | 14/40 [00:00<00:00, 42.80it/s]

...

y_true:  tensor([6, 1, 7, 8, 7, 1, 2, 7, 3, 4, 9, 0, 4, 6, 5, 6, 8, 1, 0, 8, 2, 1, 8, 2,
        5, 2, 2, 7, 8, 5, 3, 5, 3, 3, 4, 6, 4, 9, 2, 3, 9, 3, 9, 1, 3, 0, 6, 9,
        5, 2, 4, 8, 7, 1, 3, 7, 8, 4, 4, 1, 7, 5, 2, 1, 3, 3, 4, 5, 4, 6, 5, 0,
        7, 9, 2, 7, 7, 7, 2, 8, 5, 9, 3, 8, 9, 8, 9, 5, 6, 2, 3, 6, 7, 4, 6, 7,
        1, 4, 1, 5, 5, 1, 5, 8, 2, 5, 0, 9, 6, 8, 1, 8, 1, 3, 2, 6, 6, 6, 7, 4,
        6, 6, 6, 4, 2, 8, 8, 2, 9, 2, 8, 9, 7, 2, 9, 6, 3, 5, 0, 3, 6, 3, 6, 1,
        2, 5, 2, 7, 4, 6, 7, 7, 6, 5, 1, 7, 5, 4, 2, 2, 8, 8, 5, 3, 1, 0, 0, 3,
        2, 1, 3, 3, 8, 1, 3, 2, 3, 9, 5, 8, 1, 8, 2, 7, 9, 7, 5, 6, 2, 2, 3, 1,
        3, 6, 2, 1, 8, 0, 4, 9, 7, 1, 0, 6, 6, 7, 5, 8, 4, 3, 2, 7, 6, 0, 3, 2,
        9, 7, 1, 2, 3, 7, 9, 9, 2, 4, 7, 4, 6, 8, 9, 5, 0, 4, 7, 2, 9, 5, 5, 0,
        6, 3, 3, 0, 2, 7, 1, 4, 1, 6, 8, 5, 0, 2, 2, 9]) y_pred:  tensor([[1.0000e+00, 2.7988e-37, 1.7322e-25,  ..., 1.4013e-45, 1.9041e-32,
         1.4013e-45],
        [4.8322e-19, 1.0000e

 85%|████████▌ | 34/40 [00:00<00:00, 42.67it/s]

y_true:  tensor([7, 3, 4, 5, 4, 8, 3, 5, 0, 3, 9, 6, 0, 7, 9, 7, 4, 5, 1, 2, 2, 6, 7, 5,
        3, 0, 6, 5, 2, 0, 5, 6, 7, 2, 2, 3, 9, 2, 5, 6, 5, 0, 4, 8, 3, 5, 9, 4,
        3, 2, 7, 4, 1, 4, 0, 1, 9, 6, 3, 2, 8, 1, 9, 2, 1, 2, 9, 5, 0, 3, 3, 8,
        3, 2, 9, 9, 8, 4, 9, 3, 6, 1, 1, 3, 9, 6, 7, 8, 1, 5, 9, 5, 8, 6, 4, 0,
        8, 0, 5, 4, 1, 1, 7, 8, 0, 8, 9, 4, 4, 3, 9, 6, 9, 2, 4, 5, 1, 9, 0, 1,
        9, 0, 0, 0, 9, 1, 7, 2, 3, 5, 8, 6, 6, 8, 4, 8, 4, 3, 6, 3, 8, 3, 2, 8,
        8, 7, 5, 8, 0, 4, 8, 5, 7, 0, 8, 7, 9, 6, 1, 5, 7, 9, 5, 0, 0, 4, 4, 1,
        9, 5, 7, 0, 1, 3, 8, 4, 5, 1, 5, 9, 7, 8, 5, 8, 8, 6, 0, 4, 5, 2, 9, 9,
        0, 4, 1, 3, 2, 3, 4, 4, 2, 5, 4, 2, 1, 8, 1, 5, 9, 2, 8, 0, 7, 0, 6, 0,
        3, 6, 9, 4, 0, 1, 1, 0, 1, 2, 2, 8, 9, 0, 5, 5, 0, 4, 8, 7, 1, 4, 5, 8,
        6, 7, 2, 2, 6, 2, 4, 6, 4, 3, 8, 4, 5, 4, 2, 8]) y_pred:  tensor([[7.4632e-23, 1.0533e-22, 6.0749e-24,  ..., 1.0000e+00, 4.2999e-22,
         2.9426e-18],
        [1.8018e-33, 1.1620e

100%|██████████| 40/40 [00:00<00:00, 42.94it/s]

y_true:  tensor([4, 9, 4, 5, 1, 4, 6, 2, 1, 3, 3, 4, 0, 5, 4, 9, 1, 8, 5, 0, 2, 4, 8, 3,
        9, 3, 0, 5, 6, 2, 5, 8, 5, 1, 7, 2, 7, 1, 7, 1, 9, 2, 5, 1, 2, 4, 9, 0,
        1, 2, 8, 7, 6, 1, 2, 0, 8, 9, 1, 9, 0, 1, 0, 1, 4, 6, 7, 8, 8, 9, 0, 1,
        8, 5, 4, 9, 9, 2, 2, 4, 5, 7, 7, 9, 9, 7, 7, 3, 9, 0, 7, 6, 1, 5, 5, 8,
        5, 7, 5, 6, 5, 5, 5, 1, 5, 9, 4, 8, 5, 6, 3, 2, 6, 6, 7, 4, 2, 1, 0, 8,
        8, 1, 7, 9, 2, 1, 4, 2, 6, 8, 2, 3, 5, 5, 0, 5, 1, 5, 0, 9, 2, 6, 1, 0,
        9, 6, 2, 2, 2, 4, 3, 1, 7, 9, 4, 6, 4, 2, 7, 1, 1, 2, 7, 4, 6, 8, 7, 2,
        2, 6, 0, 7, 5, 0, 0, 2, 2, 5, 8, 6, 0, 8, 1, 6, 9, 4, 1, 4, 3, 6, 7, 2,
        8, 6, 6, 6, 1, 1, 7, 5, 6, 2, 5, 9, 0, 9, 8, 7, 6, 6, 6, 1, 1, 1, 8, 1,
        4, 8, 9, 7, 2, 7, 8, 0, 1, 8, 6, 0, 6, 8, 3, 8, 5, 5, 8, 7, 8, 2, 8, 6,
        5, 1, 5, 2, 3, 4, 8, 3, 6, 1, 8, 2, 7, 4, 7, 8]) y_pred:  tensor([[6.2517e-23, 4.5286e-16, 2.0721e-17,  ..., 1.1229e-21, 4.7293e-13,
         4.2895e-20],
        [6.3966e-28, 2.7315e